In [ ]:
import geopandas as gpd
import rioxarray as rio
import pandas as pd

# Load raster
raster = rio.open_rasterio("Sentinel2_NYC.tif")

# Load UHI observation points (ensure your CSV has 'Latitude' and 'Longitude' columns)
uhi_data = pd.read_csv("UHI_Observation.csv")
geo_uhi = gpd.GeoDataFrame(
    uhi_data, geometry=gpd.points_from_xy(uhi_data.Longitude, uhi_data.Latitude), crs="EPSG:4326"
)

# Extract pixel values for each UHI observation
extracted_values = []
for _, row in geo_uhi.iterrows():
    lon, lat = row.geometry.x, row.geometry.y
    pixel_values = raster.sel(x=lon, y=lat, method="nearest").values.flatten()
    extracted_values.append(pixel_values)

# Convert to DataFrame
extracted_df = pd.DataFrame(extracted_values, columns=[f"Band_{i+1}" for i in range(raster.shape[0])])
uhi_data = pd.concat([uhi_data, extracted_df], axis=1)

print("✅ Extracted Pixel Values:")
print(uhi_data.head())

# Save extracted dataset
uhi_data.to_csv("xiochen.csv", index=False)
